In [33]:
import pandas as pd
from datetime import datetime

df = pd.read_csv('taxis.csv', encoding='unicode_escape')

# Question 1a
print("Question 1a")

# Split the 'pickup' column into date and time
pickup_date = df['pickup'].str.split(' ').str[0]
pickup_time = df['pickup'].str.split(' ').str[1]

# Split the 'dropoff' column into date and time
dropoff_date = df['dropoff'].str.split(' ').str[0]
dropoff_time = df['dropoff'].str.split(' ').str[1]

# Insert these new columns to the leftmost side of the dataframe
df.insert(0, 'pickup_date', pickup_date)
df.insert(1, 'pickup_time', pickup_time)
df.insert(2, 'dropoff_date', dropoff_date)
df.insert(3, 'dropoff_time', dropoff_time)


# Print top 5 rows, dropping pickup and dropoff temporarily
print("After spliting the date and time for pick and dropoff, here's the resultant df:\n")
print(df.drop(columns=['pickup', 'dropoff']).head(5).to_string(header=True, index=False, col_space=10))

Question 1a
After spliting the date and time for pick and dropoff, here's the resultant df:

pickup_date pickup_time dropoff_date dropoff_time  passengers   distance       fare        tip      tolls      total      color     payment           pickup_zone          dropoff_zone pickup_borough dropoff_borough
 2019-03-23    20:21:09   2019-03-23     20:27:24           1       1.60        7.0       2.15        0.0      12.95     yellow credit card       Lenox Hill West   UN/Turtle Bay South      Manhattan       Manhattan
 2019-03-04    16:11:55   2019-03-04     16:19:00           1       0.79        5.0       0.00        0.0       9.30     yellow        cash Upper West Side South Upper West Side South      Manhattan       Manhattan
 2019-03-27    17:53:01   2019-03-27     18:00:25           1       1.37        7.5       2.36        0.0      14.16     yellow credit card         Alphabet City          West Village      Manhattan       Manhattan
 2019-03-10    01:23:59   2019-03-10     01:49:

In [35]:
# Question 1b - compute for the green and yellow
print("Question 1b")
# Convert pickup_time and dropoff_time to datetime
df['pickup_time'] = pd.to_datetime(df['pickup_time'], format='%H:%M:%S')
df['dropoff_time'] = pd.to_datetime(df['dropoff_time'], format='%H:%M:%S')

# Calculate the time of travel for each row and add it as a new column
df['travel_time'] = (df['dropoff_time'] - df['pickup_time']).dt.total_seconds() / 60  # This gives travel time in minutes

# convert back to time
df['pickup_time'] = pd.to_datetime(df['pickup_time'], format='%H:%M:%S').dt.time
df['dropoff_time'] = pd.to_datetime(df['dropoff_time'], format='%H:%M:%S').dt.time

# Group by car_type and compute the totals
color_agg_results = df.groupby('color').agg({
    'fare': 'sum',
    'passengers': 'sum',
    'distance': 'sum',
    'travel_time': 'sum'
})

# Convert the total travel time from minutes to "Days Hours:Minutes:Seconds"
color_agg_results['travel_time'] = pd.to_timedelta(color_agg_results['travel_time'], unit='s')
color_agg_results['travel_time'] = color_agg_results['travel_time'].astype(str)

print("The aggregated results of fare, passengers, distance and travel_time for each colour is:")
print(color_agg_results.to_string(header=True, index=False, col_space=10))


Question 1b
The aggregated results of fare, passengers, distance and travel_time for each colour is:
      fare  passengers   distance               travel_time
  13788.15        1226    3345.95 0 days 03:03:37.766666667
  70426.72        8676   16111.41 0 days 02:34:53.316666667


In [36]:
# Question 2
print("Question 2")
# Filter for cash payments and specific pickup dates
cash_df = df[(df['payment'] == 'cash') & pd.to_datetime(df['pickup_date']).dt.day.isin([10, 15, 20, 25, 30])]
# print(cash_df)

# Function to get row with max distance for each group
def get_max_distance_trip(group):
    idx = group['distance'].idxmax() 
    if pd.notna(idx):  # checks if idx is not NaN
        return group.loc[idx]
    else:
        return pd.Series({'distance': 0})

# Group by car color and pickup date, then apply the function
max_distance_df = cash_df.groupby(['color', 'pickup_date']).apply(get_max_distance_trip)

# Filter and rearrange columns
GY_cash = max_distance_df[['color', 'distance', 'pickup', 'pickup_date', 'pickup_time', 'dropoff', 'dropoff_date', 'dropoff_time', 'fare']]
GY_cash.columns = ['color', 'distance', 'pickup', 'pickup_date', 'pickup_time', 'dropoff', 'dropoff_date', 'dropoff_time', 'fare']

GY_cash = GY_cash.reset_index(drop=True)

print(GY_cash.to_string(header=True, index=False, col_space=10))

Question 2
     color   distance              pickup pickup_date pickup_time             dropoff dropoff_date dropoff_time       fare
     green      16.01 2019-03-10 00:59:49  2019-03-10    00:59:49 2019-03-10 01:36:25   2019-03-10     01:36:25       47.0
     green       4.71 2019-03-15 06:42:17  2019-03-15    06:42:17 2019-03-15 06:54:38   2019-03-15     06:54:38       15.0
     green       6.66 2019-03-20 12:04:33  2019-03-20    12:04:33 2019-03-20 12:28:07   2019-03-20     12:28:07       20.0
     green       8.80 2019-03-25 10:10:11  2019-03-25    10:10:11 2019-03-25 10:46:25   2019-03-25     10:46:25       31.0
     green      10.32 2019-03-30 09:25:32  2019-03-30    09:25:32 2019-03-30 09:54:33   2019-03-30     09:54:33       31.0
    yellow      11.52 2019-03-10 00:13:12  2019-03-10    00:13:12 2019-03-10 00:33:09   2019-03-10     00:33:09       32.5
    yellow      15.64 2019-03-15 16:45:55  2019-03-15    16:45:55 2019-03-15 17:46:47   2019-03-15     17:46:47       50.0
    y

In [37]:
# Question 3
print("Question 3")

# Group by 'pickup_date' and find the index with the maximum 'distance'
idx = GY_cash.groupby('pickup_date')['distance'].idxmax()

# Create a new dataframe with only the rows with the maximum 'distance' for each 'pickup_date'
GY_maxDist = GY_cash.loc[idx]

# Display the new DataFrame
print(GY_maxDist.to_string(header=True, index=False, col_space=10))

Question 3
     color   distance              pickup pickup_date pickup_time             dropoff dropoff_date dropoff_time       fare
     green      16.01 2019-03-10 00:59:49  2019-03-10    00:59:49 2019-03-10 01:36:25   2019-03-10     01:36:25       47.0
    yellow      15.64 2019-03-15 16:45:55  2019-03-15    16:45:55 2019-03-15 17:46:47   2019-03-15     17:46:47       50.0
    yellow      17.01 2019-03-20 17:22:33  2019-03-20    17:22:33 2019-03-20 18:07:19   2019-03-20     18:07:19       52.0
    yellow      12.12 2019-03-25 05:33:43  2019-03-25    05:33:43 2019-03-25 05:58:38   2019-03-25     05:58:38       35.5
    yellow      17.14 2019-03-30 14:56:08  2019-03-30    14:56:08 2019-03-30 15:49:32   2019-03-30     15:49:32       52.0


In [38]:
# Question 4
print("Question 4")

# Function to calculate speed
def calculate_speed(row):
    # Convert date and time columns to datetime objects
    pickup_datetime = pd.to_datetime(row['pickup_date'] + ' ' + row['pickup_time'].strftime('%H:%M:%S'))
    dropoff_datetime = pd.to_datetime(row['dropoff_date'] + ' ' + row['dropoff_time'].strftime('%H:%M:%S'))
    
    # Convert distance from kilometers to meters
    distance_m = row['distance'] * 1000
    
    # Calculate time difference in seconds
    time_difference = (dropoff_datetime - pickup_datetime).total_seconds()
    
    # Handle cases where time_difference is zero to avoid division by zero error
    if time_difference == 0:
        return 0
    
    # Calculate speed in meters per second
    speed = distance_m / time_difference
    
    return speed

# Applying the function to calculate speed
GY_maxDist['speed'] = GY_maxDist.apply(calculate_speed, axis=1)

# Display the updated DataFrame
print(GY_maxDist)


Question 4
    color  distance               pickup pickup_date pickup_time  \
0   green     16.01  2019-03-10 00:59:49  2019-03-10    00:59:49   
6  yellow     15.64  2019-03-15 16:45:55  2019-03-15    16:45:55   
7  yellow     17.01  2019-03-20 17:22:33  2019-03-20    17:22:33   
8  yellow     12.12  2019-03-25 05:33:43  2019-03-25    05:33:43   
9  yellow     17.14  2019-03-30 14:56:08  2019-03-30    14:56:08   

               dropoff dropoff_date dropoff_time  fare     speed  
0  2019-03-10 01:36:25   2019-03-10     01:36:25  47.0  7.290528  
6  2019-03-15 17:46:47   2019-03-15     17:46:47  50.0  4.282585  
7  2019-03-20 18:07:19   2019-03-20     18:07:19  52.0  6.332837  
8  2019-03-25 05:58:38   2019-03-25     05:58:38  35.5  8.107023  
9  2019-03-30 15:49:32   2019-03-30     15:49:32  52.0  5.349563  


In [49]:
# Question 5
print("Question 5")

# Step 1: Convert 'pickup_date' to datetime objects
df['pickup_date'] = pd.to_datetime(df['pickup_date'])

# Step 2: Filter the dataset for the specified conditions
filtered_df = df[
    (df['pickup_borough'] == 'Brooklyn') & 
    (df['dropoff_borough'] == 'Manhattan') & 
    (df['pickup_date'] >= '2019-03-10') & 
    (df['pickup_date'] <= '2019-03-25')
]

filtered_df = filtered_df.copy()  # Create a new copy to avoid SettingWithCopyWarning

# Step 3: Create datetime objects for 'pickup_time' and 'dropoff_time'
filtered_df['pickup_datetime'] = pd.to_datetime(filtered_df['pickup_date'].astype(str) + ' ' + filtered_df['pickup_time'].astype(str))
filtered_df['dropoff_datetime'] = pd.to_datetime(filtered_df['dropoff_date'].astype(str) + ' ' + filtered_df['dropoff_time'].astype(str))

# Compute the actual speed for each trip
def calculate_speed(row):
    distance_m = row['distance'] * 1000  # Convert distance from kilometers to meters
    time_difference = (row['dropoff_datetime'] - row['pickup_datetime']).total_seconds()
    if time_difference == 0:
        return 0
    speed = distance_m / time_difference
    return speed

filtered_df['speed'] = filtered_df.apply(calculate_speed, axis=1)

# Step 4: Calculate the mean speed for green and yellow cars
mean_speeds = filtered_df.groupby('color')['speed'].mean()

# Print the results
print(mean_speeds)


Question 5
color
green     3.357360
yellow    4.203138
Name: speed, dtype: float64


In [52]:
# Question 6
print("Question 6")

# Convert 'pickup_date' to datetime type and 'pickup_time' to time type
df['pickup_date'] = pd.to_datetime(df['pickup_date'])
df['pickup_time'] = pd.to_datetime(df['pickup_time'], format='%H:%M:%S').dt.time

# Define the start and end time
start_time = pd.to_datetime('14:30:00', format='%H:%M:%S').time()
end_time = pd.to_datetime('16:00:00', format='%H:%M:%S').time()

# Filter the dataframe for the specified date and time range
filtered_df = df[
    (df['pickup_date'].dt.date == pd.to_datetime('2019-03-17').date()) & 
    (df['pickup_time'] >= start_time) & 
    (df['pickup_time'] <= end_time)
]

# Group by 'color' and count the number of pickups for each color
color_pickup_counts = filtered_df.groupby('color')['pickup_time'].count()

# Find the color with the maximum number of pickups
most_pickups_color = color_pickup_counts.idxmax()

print(f"The color with the most pickups between 2:30 pm and 4:00 pm on March 17th was: {most_pickups_color}")


Question 6
The color with the most pickups between 2:30 pm and 4:00 pm on March 17th was: yellow


In [ ]:
# Question 7
print("Question 7")
